# Test sending data to Firefly through Flask

*Working from customReader.ipynb*


In [1]:
%load_ext autoreload
%autoreload 2

from firefly_api.reader import SimpleReader
import numpy as np
import h5py

import sys
import json
import requests
from IPython.display import IFrame

## Start the server

In [ ]:
%%bash --bg --proc bg_proc
python /Users/ageller/Visualizations/Firefly/FireflyFlaskApp.py

## Show Firefly in an IFrame

In [ ]:
url = "http://localhost:5000/combined"
IFrame(url, width=1000, height=500)

### Create some data, and get the Firefly dict

In [2]:
## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T

with h5py.File("dummy.hdf5",'w') as handle:
    group = handle.create_group('group1')
    group['Coordinates'] = coords
print("output to dummy.hdf5")

simple = SimpleReader("dummy.hdf5",write_jsons_to_disk=False)

output to dummy.hdf5
Opening 1 files and 1 particle types...
JSONdir is None, defaulting to /Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data
outputting: 
ParticleGroup: group1
Contains 8000 particles (8000 after decimation) and 0 tracked fields


## Send this data to the Flask app

*This sends the data.  (Then look back above to see it.)*

In [4]:
simple.sendDataViaFlask()

posting...data posted!


In [33]:
## alternatively, make the request yourself (literally just copying the code from sendDataViaFlask())
port = 5000
print('sending to Firefly', sys.getsizeof(simple.JSON))
requests.post(f'http://localhost:{port:d}/data_input',json=simple.JSON)

sending to Firefly 322434


<Response [200]>

*Testing with FIRE data*

In [43]:
port = 5000
fdir = '/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/snapdir_050/'
reader = SimpleReader(fdir, write_jsons_to_disk=False, decimation_factor=100)
print('sending to Firefly', sys.getsizeof(reader.JSON))
requests.post(f'http://localhost:{port:d}/data_input',json=reader.JSON)

Opening 1 files and 6 particle types...
JSONdir is None, defaulting to /Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data
outputting: 
ParticleGroup: PartType0
Contains 1066668 particles (10666 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType1
Contains 5860827 particles (58608 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType2
Contains 6175377 particles (61753 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType3
Contains 7732157 particles (77321 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType4
Contains 514118 particles (5141 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType5
Contains 1 particles (1 after decimation) and 0 tracked fields
sending to Firefly 9308911


<Response [200]>

In [34]:
fdir = '/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/snapdir_050/'
reader = SimpleReader(fdir, write_jsons_to_disk=False, decimation_factor=100)

Opening 1 files and 6 particle types...
JSONdir is None, defaulting to /Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data
outputting: 
ParticleGroup: PartType0
Contains 1066668 particles (10666 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType1
Contains 5860827 particles (58608 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType2
Contains 6175377 particles (61753 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType3
Contains 7732157 particles (77321 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType4
Contains 514118 particles (5141 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType5
Contains 1 particles (1 after decimation) and 0 tracked fields


In [7]:
#check
foo = json.loads(reader.JSON)
for fname in foo:
    print(fname, len(foo[fname]))
    if ('filenames.json' in fname):
        print(foo[fname])

/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType0000.json 426607
/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType0001.json 28455
/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType1000.json 459732
/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType1001.json 459695
/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType1002.json 459803
/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType1003.json 459626
/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType1004.json 459415
/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType1005.json 395804
/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType2000.json 425658
/Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data/DataPartType2001.json 425598
/Volumes/hi

In [8]:
#send the data to flask
port = 5000
print('sending to Firefly', sys.getsizeof(reader.JSON))
requests.post(f'http://localhost:{port:d}/data_input',json=reader.JSON)

sending to Firefly 9307723


<Response [200]>

*To kill the Firefly server.*

In [ ]:
# uncomment the next line
! ps aux | grep Firefly | awk '{print $2}' | xargs kill